In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm

In [ ]:
class Optics(tlm.Module):
    def __init__(self):
        super().__init__()

        # lens diameter in mm
        lens_diameter = 60.

        # shape of the lens surface
        self.shape = tlm.CircularArc(lens_diameter, nn.Parameter(torch.tensor(-200.)))

        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.5), inner_thickness=20.)
        
        self.optics = tlm.OpticalSequence(
            tlm.PointSourceAtInfinity(beam_diameter=0.6*lens_diameter),
            tlm.Gap(50.),
            
            self.lens,
            
            tlm.Gap(60.0), # focal length
            tlm.FocalPoint(),
        )

    def forward(self, inputs, sampling):
        return self.optics(inputs, sampling)

optics = Optics()

print(optics.shape.parameters())
for p in optics.parameters():
    print(p)

tlm.render_plt(optics)

In [ ]:
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"rays": 10},
    num_iter = 150
)

print("Final arc radius:", optics.shape.coefficients().item())

tlm.render_plt(optics)

In [ ]:
part = tlm.lens_to_part(optics.lens)
part